In [1]:
import numpy as np
import pandas as pd

#read the given text file, using separator as '|'
df = pd.read_csv('gbmdata.txt', sep = '|')
df = df.reset_index()
#remove the first unnecessary column 'Unnamed : 0'
df.drop(['Unnamed: 0'], inplace = True, axis = 1)
#remove the extra spaces in the column names
df.columns = [x.strip() for x in df.columns.values]

#packages to read ra and dec 
#convert b/w hrs and deg easily
from astropy.coordinates import SkyCoord, FK4
import astropy.units as u
#make a new column concatenating 'ra' and 'dec'
df['RD'] = df['ra'] + df['dec']
#numpy array of SkyCoord tuple (ra, dec)
radec = np.array([SkyCoord(x, frame = FK4, unit = (u.hourangle, u.deg), obstime = "J2000") for x in df['RD']])
#add two new columns with ra and dec values 
df['ra_val'] = np.array([x.ra.value for x in radec])
df['dec_val'] = np.array([x.dec.value for x in radec])

In [2]:
R = 10 #radius in degrees

In [3]:
#find nearbyGRB in a 'R' degree radius
def nearbyGRB(ra, dec, ra_val, dec_val):
    nbd = []
    for i in range(len(ra_val)):
        if ra_val[i] == ra or dec_val[i] == dec:
            continue
        if( (ra_val[i] - ra)**2 + (dec_val[i] - dec)**2 <= R**2):
            nbd.append((i, ra_val[i], dec_val[i]))
    return nbd

In [15]:
import time 
start = time.time()

#tuple containing all the nearby GRBs
#tuple : (id, ra, dec) 
# index value = center of the circle
# id corresponds to ra/dec

nbd = []
for i in range(len(df['ra_val'])):
    nbd.append(nearbyGRB(df['ra_val'][i], df['dec_val'][i], df['ra_val'], df['dec_val']))

end = time.time()
print('time elapsed : {}'.format(end - start))

time elapsed : 13.950711011886597


In [13]:
df.head()

,index,name,ra,dec,trigger_time,t90,t90_error,t90_start,fluence,fluence_error,...,flnc_spectrum_start,flnc_spectrum_stop,pflx_best_fitting_model,pflx_best_model_redchisq,flnc_best_fitting_model,flnc_best_model_redchisq,Unnamed: 33,RD,ra_val,dec_val
0,0,GRB120403857,03 41 21.2,-89 00 33,2012-04-03 20:33:58.493,4.288,1.935,-3.968,2.396400e-07,2.045800e-08,...,-2.048,0.000,pflx_comp,1.042,flnc_plaw,1.058,NaN,03 41 21.2-89 00 33,55.338333,-89.009167
1,1,GRB120227725,17 06 55.2,-88 51 36,2012-02-27 17:24:41.054,17.408,0.810,0.256,2.194900e-05,1.040300e-07,...,-1.024,19.456,pflx_comp,1.120,flnc_band,1.132,NaN,17 06 55.2-88 51 36,256.730000,-88.860000
2,2,GRB141205018,19 38 26.4,-87 34 48,2014-12-05 00:25:29.813,13.056,1.280,-6.912,1.603100e-06,4.126700e-08,...,-7.168,58.368,pflx_plaw,1.043,flnc_comp,1.024,NaN,19 38 26.4-87 34 48,294.610000,-87.580000
3,3,GRB180630467,03 15 50.1,-87 28 43,2018-06-30 11:11:54.405,12.032,1.864,-2.048,2.988000e-06,3.253700e-08,...,-2.048,10.240,pflx_comp,0.990,flnc_comp,0.969,NaN,03 15 50.1-87 28 43,48.958750,-87.478611
4,4,GRB170116238,04 51 45.6,-87 25 48,2017-01-16 05:43:15.259,9.216,3.620,-1.024,1.123100e-06,9.197500e-08,...,-1.024,1.024,pflx_comp,1.008,flnc_plaw,1.042,NaN,04 51 45.6-87 25 48,72.940000,-87.430000


In [14]:
nbd

[[(3, 48.958749999999995, -87.4786111111111)],
 [],
 [(6, 285.48999999999995, -86.0), (15, 296.39, -80.45)],
 [(0, 55.33833333333333, -89.00916666666667), (11, 41.32999999999999, -83.33)],
 [(7, 71.00999999999999, -85.93)],
 [(6, 285.48999999999995, -86.0)],
 [(2, 294.60999999999996, -87.58),
  (5, 276.57291666666663, -86.11138888888888)],
 [(4, 72.94, -87.43), (31, 66.50625, -77.26472222222222)],
 [(26, 9.479999999999999, -78.2)],
 [],
 [(12, 188.14999999999998, -82.69), (23, 187.49999999999997, -74.0)],
 [(3, 48.958749999999995, -87.4786111111111),
  (14, 31.999999999999996, -81.2),
  (25, 34.49999999999999, -78.38)],
 [(10, 186.78999999999996, -83.55), (23, 187.49999999999997, -74.0)],
 [(21, 211.21999999999997, -79.69),
  (22, 209.14, -79.1),
  (33, 215.16, -77.1),
  (44, 201.55999999999997, -76.4),
  (45, 208.47999999999996, -76.36999999999999),
  (126, 218.08999999999997, -69.51),
  (130, 214.9258333333333, -69.3611111111111),
  (140, 210.52999999999997, -68.91000000000001),
  (1